In [2]:
import numpy as np
import math

l=4 as it is length of sequence . (My name is hamza) is the sequence

d_k and d_v are dimentions initialized to 8 for illusrtrative purposes

In [3]:
l, d_k, d_v = 4,8,8
q=np.random.randn(l,d_k)
k=np.random.randn(l,d_k)
v=np.random.randn(l,d_v)

In [4]:
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[-1.34436355 -1.16481997 -0.29727278  0.25227623 -0.47776357 -0.9241674
  -0.21608085  0.60007371]
 [ 0.0847631  -2.24225177  0.52292521 -0.51078895 -0.10943865 -0.29421591
  -0.63047738 -1.13962976]
 [ 0.03225514 -1.19766006  2.1408929   1.44447214  1.73850659  0.29836894
   0.25893974  0.19151502]
 [ 0.40641467  2.45809268  1.04535716 -0.90538055  0.78173473 -0.59223458
  -0.86198921 -0.665741  ]]
K
 [[ 2.59686878e-01 -1.89382883e-01  1.07134517e+00 -2.51353453e-01
  -4.12347745e-01 -3.32750889e-01  1.66550692e+00 -3.52089610e-01]
 [ 1.11877067e+00  1.04247187e+00 -5.99701510e-01 -3.52197696e-01
   5.46609340e-01 -4.69034308e-01  2.85501242e-01  9.40605013e-01]
 [-1.56857969e+00  8.92873423e-01  5.70303848e-01 -1.21770706e+00
   1.69431683e-01 -1.66625678e-04  1.20256517e+00  4.08166157e-01]
 [-1.55094112e+00 -1.56360997e+00 -1.73763089e+00 -6.08824980e-02
  -5.28029624e-01 -3.38442417e-01 -8.92652154e-01 -5.68257178e-01]]
V
 [[ 1.01764639e+00  7.67423299e-01 -1.79409845e+00 -1.1

# Self Attention

In [9]:
selfAttention=np.matmul(q,k.T)
selfAttention

array([[-0.57705048, -1.95384555,  0.49625441,  4.8244836 ],
       [ 0.62949267, -3.55012064, -2.45662892,  3.86475355],
       [ 1.71343844, -1.94067099, -0.97386782, -3.3443139 ],
       [-0.33899972,  2.54193149,  2.08014949, -5.29971971]])

In [10]:
q.var(), k.var(), selfAttention.var()

(1.0532344645541447, 0.7702052189692108, 7.395079166165752)

In [11]:
#for normalizing and stabilizing values we divide by sqrt(d_k)
selfAttention=selfAttention/ math.sqrt(d_k)
selfAttention

array([[-0.20401815, -0.69078872,  0.17545243,  1.70571254],
       [ 0.22255927, -1.25515719, -0.86854948,  1.36639672],
       [ 0.60579197, -0.68613081, -0.34431427, -1.18239352],
       [-0.1198545 ,  0.8987085 ,  0.73544391, -1.87373387]])

In [14]:
selfAttention.var()

0.9243848957707188

# Masking

Masking is mostly done in the decoder part to ensure that the current words dont get context from the words that will be generated in the future because that will be considered cheating lol

In [15]:
mask=np.tril(np.ones((l,l)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [16]:
mask[mask==0]=-np.infty
mask[mask==1]=0
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [17]:
selfAttention=selfAttention+mask
selfAttention

array([[-0.20401815,        -inf,        -inf,        -inf],
       [ 0.22255927, -1.25515719,        -inf,        -inf],
       [ 0.60579197, -0.68613081, -0.34431427,        -inf],
       [-0.1198545 ,  0.8987085 ,  0.73544391, -1.87373387]])

# Softmax
This is done to turn vectors into probability distribution so values add upto 1 and they are interpretable as well as stable

In [18]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

In [19]:
selfAttention=softmax(selfAttention)
selfAttention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.81422742, 0.18577258, 0.        , 0.        ],
       [0.60188681, 0.16536359, 0.23274959, 0.        ],
       [0.15887164, 0.43994932, 0.37367818, 0.02750086]])

In [20]:
new_v =np.matmul(selfAttention,v)
new_v

array([[ 1.01764639,  0.7674233 , -1.79409845, -1.18994179, -0.48718051,
        -0.51441157, -1.02205241, -1.46378187],
       [ 1.30035316,  0.84495397, -1.60758256, -1.08939338, -0.51162844,
        -0.57225595, -0.86614468, -1.32657052],
       [ 0.98606185,  0.83539581, -1.53322351, -0.82689603,  0.05495859,
         0.07982152, -0.72379211, -1.15531919],
       [ 1.17033974,  0.92825448, -1.14608872, -0.49544852,  0.37871005,
         0.40545756, -0.40561645, -0.81870993]])

In [21]:
v

array([[ 1.01764639e+00,  7.67423299e-01, -1.79409845e+00,
        -1.18994179e+00, -4.87180511e-01, -5.14411568e-01,
        -1.02205241e+00, -1.46378187e+00],
       [ 2.53943594e+00,  1.18476516e+00, -7.90097281e-01,
        -6.48697172e-01, -6.18781897e-01, -8.25783563e-01,
        -1.82812716e-01, -7.25183339e-01],
       [-1.99254210e-01,  7.62952240e-01, -1.38658020e+00,
        -1.46718162e-02,  1.93559998e+00,  2.25991200e+00,
        -3.36849875e-01, -6.63241794e-01],
       [-1.24001894e+00, -9.28626420e-05,  1.70182526e-01,
        -5.64495078e-01,  1.83645303e-01,  2.18396141e-01,
        -1.34321384e+00, -7.00836419e-01]])

# Function
the function will combine all the functionalities implemented above

In [22]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention

In [25]:

values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)

Q
 [[-1.34436355 -1.16481997 -0.29727278  0.25227623 -0.47776357 -0.9241674
  -0.21608085  0.60007371]
 [ 0.0847631  -2.24225177  0.52292521 -0.51078895 -0.10943865 -0.29421591
  -0.63047738 -1.13962976]
 [ 0.03225514 -1.19766006  2.1408929   1.44447214  1.73850659  0.29836894
   0.25893974  0.19151502]
 [ 0.40641467  2.45809268  1.04535716 -0.90538055  0.78173473 -0.59223458
  -0.86198921 -0.665741  ]]
K
 [[ 2.59686878e-01 -1.89382883e-01  1.07134517e+00 -2.51353453e-01
  -4.12347745e-01 -3.32750889e-01  1.66550692e+00 -3.52089610e-01]
 [ 1.11877067e+00  1.04247187e+00 -5.99701510e-01 -3.52197696e-01
   5.46609340e-01 -4.69034308e-01  2.85501242e-01  9.40605013e-01]
 [-1.56857969e+00  8.92873423e-01  5.70303848e-01 -1.21770706e+00
   1.69431683e-01 -1.66625678e-04  1.20256517e+00  4.08166157e-01]
 [-1.55094112e+00 -1.56360997e+00 -1.73763089e+00 -6.08824980e-02
  -5.28029624e-01 -3.38442417e-01 -8.92652154e-01 -5.68257178e-01]]
V
 [[ 1.01764639e+00  7.67423299e-01 -1.79409845e+00 -1.1